In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # adiciona a pasta raiz ao PYTHONPATH

In [2]:
from pysymmetry import *

In [3]:
G = FiniteGroup(DihedralGroup(4))

In [4]:
gens = G.gens()

In [5]:
mats = [ g.matrix () for g in gens]

In [6]:
mats

[
[0 1 0 0]  [0 0 0 1]
[0 0 1 0]  [0 0 1 0]
[0 0 0 1]  [0 1 0 0]
[1 0 0 0], [1 0 0 0]
]

In [7]:
rep = representation ( gens , mats )

In [8]:
rep

Map: 
 From: Permutation Group with generators [(1,2,3,4), (1,4)(2,3)] 
 To: Matrix group over Integer Ring with 2 generators (
[0 1 0 0]  [0 0 0 1]
[0 0 1 0]  [0 0 1 0]
[0 0 0 1]  [0 1 0 0]
[1 0 0 0], [1 0 0 0]
).

In [9]:
n_irreps, irr = G.irreducible_representations(False)

In [11]:
multiplicities = []
for i in range(n_irreps):
    m_i = rep.inner_product(irr(i))        # integer multiplicity of irrep i in rep
    multiplicities.append(m_i)

In [12]:
multiplicities

[1, 0, 1, 0, 1]

In [13]:
# Report irreps present (m_i > 0) and multiplicities
present = [(i, m_i) for i, m_i in enumerate(multiplicities) if m_i > 0]
print("Irreps present (index, multiplicity):", present)

Irreps present (index, multiplicity): [(0, 1), (2, 1), (4, 1)]


In [17]:
# --- Problem 3: symmetry-adapted basis and block-diagonalization
# Build a G-equivariant matrix by averaging a random matrix over the group action
import random
d = rep.codomain().degree()
A = matrix(QQ, d, d, [random.randint(-2,2) for _ in range(d*d)])

In [18]:
A

[-1  2 -2  0]
[-1  0  2  0]
[ 0 -1  2 -2]
[-2  0 -1 -2]

In [19]:
rep.is_equivariant_to(A)

False

In [22]:
# --- Corpo algébrico K = Q(sqrt(2))
K.<s> = QuadraticField(2)

G = FiniteGroup(DihedralGroup(4))
rep = representation ( gens , mats )

d = rep.degree()

# A com coeficientes em K (usa o 's' algébrico, não simbólico)
a, b, c = K(3), K(5), K(-2)   # pode escolher racionais/algébricos
A = matrix(K, d, d, [
    a,   1,  s,   0,
    1,   b,  0,   s,
    s,   0,  c,   1,
    0,   s,  1,   a+b-c
])

elts = list(G)
Msum = sum(rep(g).change_ring(K) * A * rep(g).change_ring(K).inverse() for g in elts)
M = (K(1)/len(elts)) * Msum

# Verificação exata (sem float): comutador é a matriz zero em K
ok = True
for h in elts:
    R = rep(h).change_ring(K)
    C = R*M - M*R
    ok &= C.is_zero()
print("Equivariância exata em Q(sqrt(2)):", ok)

# Blocagem em K
iso = G.isotypic_base(rep)      # projetores racionais (permutações)
P = iso.matrix().change_ring(K) # elevação de anel
B = P.inverse() * M * P
print("Blocos (tamanhos):", [b.nrows() for b in iso.get_blocks(M)])


TypeError: unable to convert [ 3  1  s  0]
[ 1  5  0  s]
[ s  0 -2  1]
[ 0  s  1 10] to Number Field in s with defining polynomial x^2 - 2 with s = 1.414213562373095?